In [ ]:
import pandas as pd
import numpy as np
import os
from io import DEFAULT_BUFFER_SIZE
from scipy.stats import entropy
from tqdm.auto import tqdm
import pefile
import PySimpleGUI as sg


from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier

from sklearn.svm import SVC

In [ ]:
# Global variables
seed = 42
# Path to original train.csv
csv_input_train_path = 'train.csv'
# Path to original test.csv
csv_input_test_path = 'test.csv'
# Path to training data
csv_train_data_path = 'train-data.csv'
# Path to testing data
csv_test_data_path = 'test-data.csv'
# Path to training labels
csv_train_labels_path = 'train-labels.csv'
# Path to testing labels
csv_test_labels_path = 'test-labels.csv'
# Path to training samples directory
dir_train = 'train'
# Path to testing samples directory
dir_test = 'test'

In [ ]:
# Read the CSV files as DataFrames
df_train = pd.read_csv(csv_input_train_path)
df_test = pd.read_csv(csv_input_test_path)

# Rename `id` to `path` and `list` to `malicious`
df_train = df_train.rename(columns={'id': 'path', 'list': 'malicious'})
df_test = df_test.rename(columns={'id': 'path', 'list': 'malicious'})

# Change `malicious` column to bool type
df_train['malicious'] = df_train['malicious'].eq('Blacklist')
df_test['malicious'] = df_test['malicious'].eq('Blacklist')

# Change file name to relative path
df_train['path'] = df_train['path'].apply(lambda path: os.path.join(dir_train, str(path)))
df_test['path'] = df_test['path'].apply(lambda path: os.path.join(dir_test, str(path)))

# Sort by path
df_train = df_train.sort_values(by='path').reset_index(drop=True)
df_test = df_test.sort_values(by='path').reset_index(drop=True)

# Preview training data
df_train.head()

In [ ]:
# Store labels in new dataframe
df_train_labels = df_train[['path', 'malicious']].copy()
df_test_labels = df_test[['path', 'malicious']].copy()

# Save labels to CSV
df_train_labels.to_csv(csv_train_labels_path, index=False)
df_test_labels.to_csv(csv_test_labels_path, index=False)

# Store input data in new dataframe, saving only the `path` column
df_train_data = df_train[['path']].copy()
df_test_data = df_test[['path']].copy()

# Preview training data
df_train_data.head()

In [ ]:
# use magic numbers to find filetype
import magic

tqdm.pandas()
df_train_data['extension'] = df_train_data['path'].progress_apply(lambda path: magic.from_file(path, mime=True))
df_test_data['extension'] = df_test_data['path'].progress_apply(lambda path: magic.from_file(path, mime=True))

In [ ]:
print(f'# train rows:               {len(df_train_data)}')
print(f'# test rows:                {len(df_test_data)}')
print(f"# NaN train path values:    {df_train_data['path'].isna().sum()}")
print(f"# NaN test path values:     {df_test_data['path'].isna().sum()}")
print(f"# unique train path values: {df_train_data['path'].nunique()}")
print(f"# unique test path values:  {df_test_data['path'].nunique()}")

In [ ]:
def get_entropy(path):
    byte_counts = np.zeros(256, dtype=np.uint64)
    with open(path, 'rb') as file:
        while chunk := file.read(DEFAULT_BUFFER_SIZE):
            byte_counts += np.bincount(np.frombuffer(chunk, dtype=np.uint8), minlength=256).astype(np.uint64)
    file_size = os.path.getsize(path)
    return entropy(byte_counts / file_size, base=2) if file_size else 0

In [ ]:
# Store the Shannon entropies of the files
df_train_data['entropy'] = df_train_data['path'].progress_apply(get_entropy)
df_test_data['entropy'] = df_test_data['path'].progress_apply(get_entropy)

# Preview training data
df_train_data.head()

In [ ]:
def parse_pe(path):
    d = dict()
    try:
        d = pefile.PE(path).dump_dict()
        d.pop('LOAD_CONFIG', None)
        d.pop('TLS', None)
        d['Parsing Warnings'] = 'Parsing Warnings' in d
        _keys = list(d.keys())
        for key in _keys:
            if isinstance(d[key], list):
                d.pop(key, None)

    except Exception as e:
        print(f'Error processing {path}: {e}')
        d['Parsing Warnings'] = True

    return d

In [ ]:
# Make copies of the dataframes because the next operation takes a long time
_temp_df_train_data = df_train_data.copy()
_temp_df_test_data = df_test_data.copy()

# Get the portable executable file information
# This excludes LOAD_CONFIG and TLS data and columns whose values are lists
_temp_train = _temp_df_train_data['path'].progress_apply(parse_pe)
_temp_test = _temp_df_test_data['path'].progress_apply(parse_pe)

In [ ]:
# Add the portable executable file information to the data
df_train_data = pd.concat([_temp_df_train_data, pd.json_normalize(_temp_train)], axis=1)
df_test_data = pd.concat([_temp_df_test_data, pd.json_normalize(_temp_test)], axis=1)

# Replace all NaN values with 0
df_train_data = df_train_data.fillna(0)
df_test_data = df_test_data.fillna(0)

# Exclude columns with 2 or fewer unique values
_cols = df_train_data.columns[df_train_data.nunique() <= 2].tolist()
df_train_data = df_train_data.drop(columns=_cols)
df_test_data = df_test_data.drop(columns=_cols)

# Exclude columns that end with 'Offset' (this removes a lot of useless data)
_cols = [_col for _col in df_train_data.columns if _col.endswith('Offset')]
df_train_data = df_train_data.drop(columns=_cols)
df_test_data = df_test_data.drop(columns=_cols)

In [ ]:
# Preview training data
df_train_data.head()

In [ ]:
# Some of the columns' data types are numerical, but many are strings (object)
# We may want to one-hot encode the columns whose data types are not numeric
# https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics
df_train_data.dtypes

In [ ]:
# Or... we can just drop non-numeric data for simplicity
_cols = df_train_data.select_dtypes(exclude=np.number).drop(columns='path')
df_train_data = df_train_data.drop(columns=_cols)
df_test_data = df_test_data.drop(columns=_cols)

# All of the columns should be numeric except for `path`
df_train_data.dtypes

In [ ]:
# Checkpoint progress
df_train_data.to_csv(csv_train_data_path, index=False)
df_test_data.to_csv(csv_test_data_path, index=False)

*** IMPORTANT ***

The cells above will generate the csv files needed for training and testing. After running the cells above once, you don't need to run them again.

In [ ]:
# Load save state
df_train_data = pd.read_csv(csv_train_data_path)
df_test_data = pd.read_csv(csv_test_data_path)

df_train_labels = pd.read_csv(csv_train_labels_path)
df_test_labels = pd.read_csv(csv_test_labels_path)

In [ ]:
# Z-score normalize all the data (except for `path`, which isn't numeric)
scaler = StandardScaler()
x_train = scaler.fit_transform(df_train_data.drop(columns=['path']))
x_test = scaler.transform(df_test_data.drop(columns=['path']))

# Preview the input data
x_train[:5]

In [ ]:
# Get the training labels
y_train = np.array(df_train_labels['malicious'])
y_test = np.array(df_test_labels['malicious'])

# Verify that the training and testing datasets are balanced
print(df_train_labels['malicious'].value_counts(), end='\n\n')
print(df_test_labels['malicious'].value_counts())

In [ ]:
models = {
    'Adaptive Boosting Classifier': AdaBoostClassifier(random_state=seed),
    'GBM': GradientBoostingClassifier(random_state=seed),
    'Histogram-based GBM': HistGradientBoostingClassifier(random_state=seed),
    'Random Forest Classifier': RandomForestClassifier(random_state=seed),
    'MLP': MLPClassifier(max_iter=1000, random_state=seed),
    'C-SVC': SVC(random_state=seed),
}

for name, model in models.items():
    # Train the model
    model = model.fit(x_train, y_train)
    # Test the model
    y_pred = model.predict(x_test)
    # Print the results of the test
    print(f'{name}: {accuracy_score(y_test, y_pred)}')

In [ ]:
def scan_file(filename):
    #grab file object, give to AI, return either malware or not
    output = filename #placeholder line
    return output

In [ ]:
layout = [[sg.Text('What file do you want to scan?')], 
          [sg.Input(key='-IN-', enable_events=True), sg.FileBrowse(target='-IN-', initial_folder='Downloads')],
          [sg.Text(key='-OUT-', size=(50,10), text='')],
          [sg.Button('Exit')]]
window = sg.Window('File Scan', layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    if event == '-IN-':
        filename = values[event]
        output = scan_file(filename)
        window['-OUT-'].update(output)

window.close()